# 栄養パターンに基づく口腔・上部消化管がん分類モデルの構築および評価１

In [ ]:
%reset -f

# 概要

## パッケージインストール

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from sklearn.linear_model import LinearRegression
!pip install statsmodels
from statsmodels.duration.hazard_regression import PHReg
!pip install factor_analyzer
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from decimal import Decimal, ROUND_HALF_UP
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
import time
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import warnings
from IPython.display import display

## データセットの読み込みと処理

J-MICC_ベースライン・死亡・転出データ_241202はall_241101.csv。
まず欠損値を定義し、（99999が欠損のものもあれば、999や888が欠損のものもある、0や9は欠損の時もあればそうでもないときもあるので、あとで個別に定義）
データの読み込みはpd.read_csvを使う。

In [ ]:
missing_values = [99999, 999, 888]
# データの読み込み
df=pd.read_csv('ベースラインメタデータ', na_values=missing_values)

In [ ]:
warnings.filterwarnings('ignore')
#2行目をヘッダーとして取り出す
new_header = df.iloc[1]
#それ以降の行をデータとして使う
df2 = df[2:]
#ヘッダーを設定しなおす
df2.columns = new_header
display(df2)

In [ ]:
df2.head()

In [ ]:
# 研究IDをprefixとidに分ける
kamashi_a= df2['研究ID'].str.extract(r'(研究ID)(\d+)')
kamashi_a.columns = ['prefix', 'id1']
df2 = pd.concat([df2,kamashi_a], axis=1)
# データの情報を損なわないようにjを付与
df2['id1'] = 'j' + df2['id1'].astype(str)
display(df2)

0行目が日本語の名前、1行目が英語の名前、2行目からデータが始まっている。1行目を変数の名前として使用、2行目からデータとして使用した新しいdf2というデータフレームを作成。

In [ ]:
#用いる変数を選択する。列名は例
df2_selected = df2[['id1', '性別', '場所',...]]

In [ ]:
df2_selected.head()

「nutrients_241101.csv」を読み込む。やり方は同じ。

In [ ]:
nut_df=pd.read_csv('栄養素データ')

In [ ]:
nut_df.head()

栄養のデータは0行目が日本語の名前、1行目が英語の名前、２行目は単位となっている。1行目を変数名（header）として使用し、3行目からデータとして使用したnut_df2という新しいデータフレームを作成する。

In [ ]:
#１行目をヘッダーとして取り出す
new_header_nut = nut_df.iloc[1]

#それ以降の行をデータとして使う
nut_df2 = nut_df[3:]

#ヘッダーを設定しなおす
nut_df2.columns = new_header_nut

print(nut_df2)

In [ ]:
#１行目をヘッダーとして取り出す
new_header_nut = nut_df.iloc[1]

#それ以降の行をデータとして使う
nut_df2 = nut_df[3:]

#ヘッダーを設定しなおす
nut_df2.columns = new_header_nut

print(nut_df2)

使用する栄養素のみを選んだnut_df2_selectedを作成する。

In [ ]:
print(nut_df2.columns) 

In [ ]:
#用いる変数を選択する。列名は例
nut_df2_selected = nut_df2[['id','総エネルギー量', 'タンパク質',...]]

In [ ]:
display(nut_df2) 

In [ ]:
nut_df2_selected.head()

「cancer-2019.csv」を読み込む。やり方は同じ。

In [ ]:
missing_values = [99999, 999, 888]
# データの読み込み がん
can_df=pd.read_csv('がん罹患データ', na_values=missing_values)

In [ ]:
#2行目をヘッダーとして取り出す
new_header = can_df.iloc[1]
#それ以降の行をデータとして使う
can_df2 = can_df[2:]
#ヘッダーを設定しなおす
can_df2.columns = new_header
print(can_df2)

使用する変数 
'id', 'rikan_yn', 'rikan_followup', 'ca1_icdo3t',"lastfupdtc","consentdtc","ca1_diagdtc" 

In [ ]:
#独自研究番号をprefixとidに分ける
# jmiccidをprefixとidに分ける
# データの情報を損なわないようにjを付与
kamashi_a= can_df2['研究ID'].str.extract(r'(研究ID)(\d+)')
kamashi_a.columns = ['prefix', 'id1']
can_df2 = pd.concat([can_df2,kamashi_a], axis=1)
can_df2['id1'] = 'j' + can_df2['id1'].astype(str)
display(can_df2)

In [ ]:
can_df2_selected = can_df2[['id1', 'がん罹患', ...]]

## データセットのマージ

df2_selectedとnut_df2_selectedをマージした、merged_dfというデータフレームを作成する。

In [ ]:
merged_df = (
    pd.merge(df2_selected, nut_df2_selected, on='id1', how='inner')
      .merge(can_df2_selected, on='id1', how='inner')
)


In [ ]:
print(merged_df)

In [ ]:
merged_df.to_csv("merged_can_df.csv", index=False)

## データの型の決定

In [ ]:
# 読み込みたい列とそのデータ型を辞書で定義
column_types = {
    # ----------------------------------------------------
    # int64 で指定する項目
    # ----------------------------------------------------
    'sex': 'Int64',
    'age': 'Int64',
    'site': 'Int64',
    'edu': 'Int64',
    'rikan_yn': 'Int64',
    'rikan_followup': 'Int64',
    
    # ----------------------------------------------------
    # float64 で指定する項目
    # ----------------------------------------------------
    'height_e': 'float64',
    'weight_e': 'float64',
    'height_q': 'float64',
    'weight_q': 'float64',
    'ene': 'float64',
    'prot': 'float64',
    'carbo': 'float64',
    'Na': 'float64',
    'K': 'float64',
    'Ca': 'float64',
    'Fe': 'float64',
    'carotene': 'float64',
    'retiEQ': 'float64',
    'VD': 'float64',
    'VE': 'float64',
    'VB1': 'float64',
    'VB2': 'float64',
    'folate': 'float64',
    'VC': 'float64',
    'SFA': 'float64',
    'MUFA': 'float64',
    'chol': 'float64',
    'SDF': 'float64',
    'IDF': 'float64',
    'n3PUFA': 'float64',
    'n6PUFA': 'float64',
    
    # ----------------------------------------------------
    # str (string) で指定する項目
    # ----------------------------------------------------
    'ca1_icdo3t': 'string', # string は Pandas 推奨の文字列型
    'ca1_diagdtc': 'string',
    'area': 'string'
}

In [ ]:
file_path = 'merged_can_df.csv' 
# 読み込まれたデータ型の確認
merged_df=pd.read_csv('merged_can_df.csv',header=0,dtype=column_types)
display(merged_df)

#### マージしたデータセットをcsvファイルとして書き出す。

In [ ]:
merged_df.head()

In [ ]:
merged_df.dtypes

## 新たなデータフレーム作成およびデータ処理

### 身体活動量の計測

In [ ]:
#9 or 0が欠損となっている変数の9や0を欠損値とする。
columns_to_replace = ['edu','exer_freq_light', 'exert_time_light', 'exer_freq_mod', 'exer_time_mod', 'exer_freq_heavy', 'exert_time_heavy', 
                      'dr_yn', 'dr_freq_sake', 'dr_freq_liq', 'dr_freq_chu', 'dr_freq_beerl', 'dr_freq_beerm', 'dr_freq_beer350', 'dr_freq_beer250', 'dr_freq_whis', 'dr_freq_whiw', 'dr_freq_wine', 
                      'dr_qty_sake', 'dr_qty_liq', 'dr_qty_chu', 'dr_qty_beerl', 'dr_qty_beerm', 'dr_qty_beer350', 'dr_qty_beer250', 'dr_qty_whis', 'dr_qty_whiw', 'dr_qty_wine', 
                      'sm_yn', 'pihd', 'pstroke', 'cancer1', 'cancer2']

# まず対象列を一括で数値型に変換
merged_df[columns_to_replace] = merged_df[columns_to_replace].apply(pd.to_numeric, errors='coerce')

# replace を適用
merged_df[columns_to_replace] = merged_df[columns_to_replace].replace([9, 0], pd.NA)

In [ ]:
# 変換ルール(頻度)
value_map_freq = {
    1: 0,
    2: 0.5,
    3: 1.5,
    4: 3.5,
    5: 6
}

# 対象の列名をリストで指定
columns_to_map_freq = ['exer_freq_light', 'exer_freq_mod', 'exer_freq_heavy']

# 各列に対して map を適用して、新しい列を追加
for col in columns_to_map_freq:
    merged_df[col + 'r'] = merged_df[col].map(value_map_freq)

In [ ]:
# 変換ルール(時間)
value_map_time = {
    1: 0.25,
    2: 0.75,
    3: 1.5,
    4: 2.5,
    5: 3.5,
    6: 4.0
}

# 対象の列名をリストで指定
columns_to_map_time = ['exert_time_light', 'exer_time_mod', 'exert_time_heavy']

# 各列に対して map を適用して、新しい列を追加
for col in columns_to_map_time:
    merged_df[col + 'r'] = merged_df[col].map(value_map_time)

In [ ]:
# exer_freq_lightrとexert_time_lightrを掛け算して新しい列weakを作成
merged_df['weak'] = 3.4*merged_df['exer_freq_lightr'] * merged_df['exert_time_lightr']
merged_df.loc[merged_df['exer_freq_light'] == 1, 'weak'] = 0
merged_df['moderate'] = 7*merged_df['exer_freq_modr'] * merged_df['exer_time_modr']
merged_df.loc[merged_df['exer_freq_mod'] == 1, 'moderate'] = 0
merged_df['intense'] = 10*merged_df['exer_freq_heavyr'] * merged_df['exert_time_heavyr']
merged_df.loc[merged_df['exer_freq_heavy'] == 1, 'intense'] = 0

In [ ]:
merged_df['methw'] = merged_df['weak'] + merged_df['moderate'] + merged_df['intense']

### 教育データの調節

In [ ]:
# educ 列の初期化
merged_df['educ'] = np.nan

# 学歴の再カテゴリ分け
merged_df.loc[merged_df['edu'] == 1, 'educ'] = 1
merged_df.loc[merged_df['edu'].isin([2, 3, 4]), 'educ'] = 2
merged_df.loc[merged_df['edu'].isin([5, 6]), 'educ'] = 3
merged_df.loc[merged_df['edu'].isin([0, 7, 9]), 'educ'] = 4

### BMIの列の作成

In [ ]:
merged_df['bmiq'] = merged_df['weight_q']/((merged_df['height_q']/100)*(merged_df['height_q']/100))

### エネルギー過多データの削除

In [ ]:
# エネルギー摂取量が高すぎたり、低すぎたりしたら欠損とする。(NaNに置き換える)
merged_df['ene'] = merged_df['ene'].where((merged_df['ene'] < 4000) & (merged_df['ene'] >= 1000), np.nan)
merged_df['ene'].isna().sum()

### 追跡データの抽出

In [ ]:
#まずは日付型に変換
merged_df['追跡開始日'] = pd.to_datetime(merged_df['追跡開始日'], errors='coerce')
merged_df['lastfupdtc'] = pd.to_datetime(merged_df[''], errors='coerce')
merged_df['consentdtc'] = pd.to_datetime(merged_df['consentdtc'], errors='coerce')

In [ ]:
# ca1_diagdtcが欠損でないところだけ、lastfupdtcをca1_diagdtcに置き換えたlastdateを作成
merged_df['lastdate'] = merged_df['ca1_diagdtc'].where(merged_df['ca1_diagdtc'].notna(), merged_df['lastfupdtc'])

In [ ]:
merged_df[['lastfupdtc','ca1_diagdtc', 'lastdate']].head(30)

In [ ]:
# 差を計算して日数に変換
merged_df['period'] = (merged_df['lastdate'] - merged_df['consentdtc']).dt.days

In [ ]:
merged_df['flwtimey'] = merged_df['period']/365.25
#追跡期間1年以内の人と、追跡期間欠損の人を除外
merged_df['flwtimey'] = merged_df['flwtimey'].where((merged_df['flwtimey'] > 1), np.nan)
merged_df['flwtimey'].isna().sum()

In [ ]:
# 除外対象の条件（NaN または 0 以下）
exclude_by_flwtime = (merged_df['flwtimey'].isna()) | (merged_df['flwtimey'] <= 1)

# 除外される人数を表示
exclude_by_flwtime.sum()

In [ ]:
merged_df['rikan_followup'].value_counts(dropna=False)

### siteとareaのデータ処理

In [ ]:
# site を数値に変換
merged_df['site'] = pd.to_numeric(merged_df['site'], errors='coerce')
merged_df['area'] = pd.to_numeric(merged_df['area'], errors='coerce')
# educ 列の初期化
merged_df['sitec'] = np.nan

#siteをsitecに
merged_df.loc[merged_df['site'] == 1, 'sitec'] = 1
merged_df.loc[merged_df['site'] == 2, 'sitec'] = 2
merged_df.loc[merged_df['site'] == 3, 'sitec'] = 3
merged_df.loc[(merged_df['site'] == 4)&(merged_df['area'] == 1), 'sitec'] = 4
merged_df.loc[(merged_df['site'] == 4)&(merged_df['area'] == 4), 'sitec'] = 5
merged_df.loc[merged_df['site'] == 5, 'sitec'] = 6
merged_df.loc[merged_df['site'] == 6, 'sitec'] = 7
merged_df.loc[merged_df['site'] == 7, 'sitec'] = 8
merged_df.loc[merged_df['site'] == 8, 'sitec'] = 9
merged_df.loc[merged_df['site'] == 9, 'sitec'] = 10
merged_df.loc[merged_df['site'] == 10, 'sitec'] = 11
merged_df.loc[merged_df['site'] == 11, 'sitec'] = 12
merged_df.loc[merged_df['site'] == 12, 'sitec'] = 13
merged_df.loc[merged_df['site'] == 13, 'sitec'] = 14

In [ ]:
#がん罹患追跡Flag=1の人を残す。
merged_df1 = merged_df[(merged_df['rikan_followup']==1)]
display(merged_df1)

In [ ]:
merged_df1.describe()

### 早期発症および既往歴のあるデータの削除

In [ ]:
# 除外対象の条件（NaN または 1年 以下）→早期発症例を除くことで、逆因果関係の影響を少なく（口腔がんになったから不健康な食事を始めるなど）
exclude_by_flwtime = (merged_df1['flwtimey'].isna()) | (merged_df1['flwtimey'] <= 1)
# 除外される人数を表示
exclude_by_flwtime.sum()

In [ ]:
#追跡期間による除外
merged_df2 = merged_df1[(merged_df1['flwtimey'].notna()) & (merged_df1['flwtimey'] > 1)]
display(merged_df2)

In [ ]:
merged_df2.describe()

In [ ]:
#既往歴による除外
#心筋梗塞・狭心症
merged_df3 = merged_df2.dropna(subset=['pihd'])[~merged_df2['pihd'].isin([2, 3])]

In [ ]:
#脳卒中
merged_df3 = merged_df3.dropna(subset=['pstroke'])[~merged_df2['pstroke'].isin([2, 3])]

In [ ]:
#がん
# 条件1: cancer1 in [2,3,4] または cancer2 in [2,3]
cond1 = merged_df3['cancer1'].isin([2, 3, 4]) | merged_df3['cancer2'].isin([2, 3])

In [ ]:
# 条件2: cancer1 と cancer2 がどちらも NaN
cond2 = merged_df3['cancer1'].isna() & merged_df3['cancer2'].isna()

In [ ]:
# 両方の条件を満たす行を除外
merged_df3 = merged_df3[~(cond1 | cond2)]
display(merged_df3)

In [ ]:
merged_df3.describe()

## がん判別コード抽出

In [ ]:
#エネルギー摂取量による除外
#ene
merged_df5 = merged_df3.dropna(subset=['ene'])
merged_df5.describe()

In [ ]:
merged_df5.to_csv("merged_can_df5.csv", index=False)

In [ ]:
# データの読み込み
merged_df5=pd.read_csv('merged_can_df5.csv')
pd.set_option('display.max_columns', None)
display(merged_df5)
pd.reset_option('display.max_columns')

In [ ]:
# 欠損対応：NaN → 空文字 "" にして startswith を安全に使う
ca1_icdo3t_clean = merged_df5['ca1_icdo3t'].fillna("")
merged_df5["stomach"] = ca1_icdo3t_clean.str.startswith("C16").astype(int)
merged_df5["colon"] = ca1_icdo3t_clean.str.startswith(("C18", "C19", "C20", "C21")).astype(int)
merged_df5["liver"] = ca1_icdo3t_clean.str.startswith("C22").astype(int)
merged_df5["livbil"] = ca1_icdo3t_clean.str.startswith(("C22", "C23", "C24")).astype(int)
merged_df5["pancreas"] = ca1_icdo3t_clean.str.startswith(("C25")).astype(int)
merged_df5["lung"] = ca1_icdo3t_clean.str.startswith("C34").astype(int)
merged_df5["kidney"] = ca1_icdo3t_clean.str.startswith(("C64", "C65")).astype(int)
merged_df5["breast"] = ca1_icdo3t_clean.str.startswith("C50").astype(int)
merged_df5["endo"] = ca1_icdo3t_clean.str.startswith("C54").astype(int)
merged_df5["prostate"] = ca1_icdo3t_clean.str.startswith("C61").astype(int)

In [ ]:
# 口腔上部消化管がん (C00〜C169) のフラグを作成
merged_df5["oral"] = ca1_icdo3t_clean.str.startswith(
    tuple([f"C0{i}" for i in range(7)])
).astype(int)
merged_df5["oral"] = ca1_icdo3t_clean.str.match(r"^C(0[0-9][0-9]|1[0-6][0-9])$").astype(int)

In [ ]:
df=merged_df5
condition_to_drop = (df['rikan_yn'] == 1) & (df['oral'] == 0)
# 削除したい条件の逆（残したい行）を選択し、データフレームを更新
merged_df5 = df[~condition_to_drop]
merged_df5['rikan_yn'].value_counts(dropna=False)

In [ ]:
merged_df5['oral'].value_counts(dropna=False)

In [ ]:
merged_df5_a=merged_df5.iloc[:,47:69].copy()
display(merged_df5_a)
display(merged_df5)

## 栄養データの調節¶

### 歪みの計測

In [ ]:
 # 歪度を計算するためのライブラリ
df = merged_df5
# 歪度を計算したい列のリスト
skew_columns = [
    'ene', 'prot', 'carbo', 'Na', 'K', 'Ca', 'Fe', 'carotene', 'retiEQ', 'VD', 'VE', 
    'VB1', 'VB2', 'folate', 'VC', 'SFA', 'MUFA', 'chol', 'SDF', 'IDF', 'n3PUFA', 'n6PUFA'
]
# 結果を格納するための辞書
skewness_results = {}
# 各列についてループ処理を行い、歪度を計算
for col in skew_columns:
    # 1. 欠損値を除外: .dropna()
    # 2. 歪度を計算: stats.skew(...)
    # 3. 結果を辞書に格納
    if col in df.columns:
        skew_value = stats.skew(df[col].dropna())
        skewness_results[col] = skew_value
    else:
        print(f"注意: 列 '{col}' はデータフレームに存在しません。スキップします。")
# 結果を Pandas Series または DataFrame として表示
skew_series = pd.Series(skewness_results, name='Skewness')
print("--- 各栄養素列の歪度 ---")
print(skew_series)

In [ ]:
def best_skewness_transform(series, threshold=0.5):
    """
    与えられたSeriesに対し、複数の変換を試行し、最も歪度 (Skewness) が0に近い変換を特定する。

    Parameters:
    - series (pd.Series): 変換対象のデータ列 (欠損値は事前に処理済みであること)。
    - threshold (float): Box-Cox変換を適用する際のデータ値の最小閾値。
                         (データが正の値のみである必要があるため)

    Returns:
    - dict: 最適な変換方法 ('method') とその歪度 ('skewness') を含む辞書。
    """
    
    # 欠損値を除去
    data = series.dropna()

    # データが空の場合は処理をスキップ
    if data.empty or len(data) < 3:
        return {'method': 'No Transform (Insufficient Data)', 'skewness': np.nan}
        
    # すべての変換結果を格納するリスト
    results = []

    # --- 1. 変換なし ---
    results.append({
        'method': 'No Transform',
        'skewness': stats.skew(data)
    })

    # --- 2. 基本的な変換 (Log, Sqrt, Cbrt) ---
    # データが正の値のみの場合に適用可能
    if (data > 0).all():
        # Log 変換
        results.append({
            'method': 'Log (np.log)',
            'skewness': stats.skew(np.log(data))
        })
        # 平方根 (Sqrt) 変換
        results.append({
            'method': 'Sqrt (np.sqrt)',
            'skewness': stats.skew(np.sqrt(data))
        })
        # 立方根 (Cbrt) 変換
        results.append({
            'method': 'Cbrt (np.cbrt)',
            'skewness': stats.skew(np.cbrt(data))
        })
        
    # --- 3. Left-Skewed への対応 (Right-Skewedに変換後 Log) ---
    # データが負の歪み (左裾が長い) の場合に、まず反転させてから Log 変換
    if stats.skew(data) < -0.1: # 負の歪みが顕著な場合
        # 1. データ反転: max(x) + 1 - x
        max_val = data.max()
        inverted_data = max_val + 1 - data
        
        # 2. 反転したデータを Log 変換
        # 反転後のデータは正の値のみであると期待される
        if (inverted_data > 0).all():
             results.append({
                'method': 'Invert then Log',
                'skewness': stats.skew(np.log(inverted_data))
            })

    # --- 4. Box-Cox 変換 ---
    # データが正の値のみで、ある程度のサイズがある場合に適用可能
    if (data > threshold).all() and len(data) >= 5:
        try:
            # Box-Cox変換 (最適な λ を自動探索)
            transformed_data, _ = stats.boxcox(data)
            results.append({
                'method': 'Box-Cox (Optimal Lambda)',
                'skewness': stats.skew(transformed_data)
            })
        except Exception as e:
            # Box-Coxが失敗した場合（データが極端な場合など）
            print(f"Box-Cox failed for {series.name}: {e}")

    # --- 最適な変換の特定 ---
    if not results:
         return {'method': 'No suitable transform found', 'skewness': stats.skew(data)}

    # 歪度の絶対値が最も小さいものを選択
    best_result = min(results, key=lambda x: abs(x['skewness']) if not np.isnan(x['skewness']) else np.inf)

    return best_result

In [ ]:
df = merged_df5
# 歪度を計算したい列のリスト
skew_columns = [
    'ene', 'prot', 'carbo', 'Na', 'K', 'Ca', 'Fe', 'carotene', 'retiEQ', 'VD', 'VE', 
    'VB1', 'VB2', 'folate', 'VC', 'SFA', 'MUFA', 'chol', 'SDF', 'IDF', 'n3PUFA', 'n6PUFA'
]

# 結果を格納するための DataFrame
optimal_transform_df = pd.DataFrame(columns=['Original Skewness', 'Best Transform Method', 'New Skewness'])

for col in skew_columns:
    if col in df.columns:
        # 元の歪度を計算
        original_skew = stats.skew(df[col].dropna())
        
        # 最適な変換を探索
        result = best_skewness_transform(df[col])
        
        # 結果を DataFrame に追加
        optimal_transform_df.loc[col] = [
            original_skew, 
            result['method'], 
            result['skewness']
        ]
    else:
        print(f"注意: 列 '{col}' はデータフレームに存在しません。スキップします。")

# 結果を表示
print("\n--- 各栄養素列の最適変換結果 ---")
display(optimal_transform_df.sort_values(by='New Skewness', key=np.abs))

In [ ]:
def best_skewness_transform(series, threshold=0.5):
    pass
def get_transformed_data(series, method, threshold=0.5):
    """特定の変換方法を適用し、変換後の Series を返す関数。"""
    data = series.dropna()
    
    if method == 'No Transform':
        # 変換なしの場合、そのまま返す (欠損値の扱いは元のまま)
        return series
    
    elif method == 'Log (np.log)':
        # Log 変換: np.log(x)
        return np.log(series)
    
    elif method == 'Sqrt (np.sqrt)':
        # 平方根 (Sqrt) 変換: np.sqrt(x)
        return np.sqrt(series)

    elif method == 'Cbrt (np.cbrt)':
        # 立方根 (Cbrt) 変換: np.cbrt(x)
        return np.cbrt(series)

    elif method == 'Invert then Log':
        # Invert then Log 変換: np.log(max(x) + 1 - x)
        max_val = data.max()
        # 元の Series に適用（NaNも考慮）
        inverted = max_val + 1 - series
        return np.log(inverted)

    elif method == 'Box-Cox (Optimal Lambda)':
        # Box-Cox 変換
        # Box-Coxは欠損値を許容しないため、一旦欠損値を除去して計算し、元のインデックスに戻す
        if (data > threshold).all():
            transformed_array, _ = stats.boxcox(data)
            # 変換結果を元のインデックスに再結合し、NaNを再導入
            transformed_series = pd.Series(transformed_array, index=data.index)
            return series.combine_first(transformed_series) # 欠損値を保持しながら結合
        else:
            print(f"警告: {series.name} にBox-Coxを適用できませんでした。元のデータを返します。")
            return series
    else:
        # その他の場合は元のデータを返す
        return series


# 元のデータフレーム名
df_original = merged_df5_a.copy()

# 変換対象の列リスト 
skew_columns = [
    'ene', 'prot', 'carbo', 'Na', 'K', 'Ca', 'Fe', 'carotene', 'retiEQ', 'VD', 'VE', 
    'VB1', 'VB2', 'folate', 'VC', 'SFA', 'MUFA', 'chol', 'SDF', 'IDF', 'n3PUFA', 'n6PUFA'
]

# 変換後のデータフレームを作成 
df_transformed = df_original.copy()

# 最適な変換結果 DataFrame (optimal_transform_df) を使用してループ
for col in skew_columns:
    if col in df_original.columns:
        # 1. 最適な変換方法を取得
        try:
            best_method = optimal_transform_df.loc[col, 'Best Transform Method']
        except KeyError:
            print(f"エラー: 列 '{col}' の変換方法が optimal_transform_df に見つかりません。スキップします。")
            continue

        # 2. データを変換
        transformed_series = get_transformed_data(df_original[col], best_method)
        
        # 3. 新しいデータフレームに変換結果を格納
        # 新しい列名として「元の列名_trans」のようにサフィックスを追加
        new_col_name = f"{col}_trans"
        df_transformed[new_col_name] = transformed_series
        
        print(f"列 '{col}' は '{best_method}' で変換され、'{new_col_name}' として保存されました。")
    
# 変換されたデータフレームの確認
print("\n--- 変換後のデータフレームの最初の5行 (元の列と変換後の列を比較) ---")
display(df_transformed.head())

In [ ]:
merged_df5_b=df_transformed.iloc[:,22:44]
display(merged_df5_b)
merged_df5_b['ene_trans'].describe()

### エネルギー調節

In [ ]:
X=merged_df5_b[['ene_trans']]
y_columns=['prot_trans', 'carbo_trans', 'Na_trans', 'K_trans', 'Ca_trans', 'Fe_trans', 'carotene_trans', 'retiEQ_trans', 'VD_trans', 'VE_trans', 'VB1_trans', 'VB2_trans', 'folate_trans', 'VC_trans', 'SFA_trans', 'MUFA_trans', 'chol_trans', 'SDF_trans', 'IDF_trans', 'n3PUFA_trans', 'n6PUFA_trans']

# 結果保存用
models = {}
coefficients = {}

for y_col in y_columns:
    y = merged_df5_b[y_col]
    model = LinearRegression().fit(X, y)
    models[y_col] = model
    coefficients[y_col] = {
        'intercept': model.intercept_,
        'coef_X1': model.coef_[0]
    }
    
# 回帰係数をDataFrameで表示
coef_df = pd.DataFrame(coefficients).T
print(coef_df)

In [ ]:
# 調整後の値を計算・保存
adjusted_values = {}
mean= merged_df5_b["ene_trans"].mean()
for y_col in y_columns:
    coef = coefficients[y_col]['coef_X1']
    y = merged_df5_b[y_col]
    x = merged_df5_b['ene_trans']
    adjusted = y - (x - mean) * coef
    adjusted_values[f'{y_col}_adj'] = adjusted 

# DataFrameとしてまとめて表示
adjusted_df = pd.DataFrame(adjusted_values)
display(adjusted_df)

In [ ]:
print(adjusted_df.dtypes)

In [ ]:
pd.set_option('display.max_columns', None)
display(merged_df5_b)
pd.reset_option('display.max_columns')

In [ ]:
pd.set_option('display.max_columns', None)
display(merged_df5)
pd.reset_option('display.max_columns')

In [ ]:
merged_df5.to_csv("merged_df5.csv", index=False)
merged_df5_b.to_csv("merged_df5_b.csv", index=False)
merged_df5_a.to_csv("merged_df5_a.csv", index=False)
adjusted_df.to_csv("adjusted_df.csv", index=False)